In [1]:
import pandas as pd
import numpy as np

def import_data(filename):
    df_ = pd.DataFrame()
    d = np.loadtxt(filename)
    df = pd.DataFrame(d, columns=['YYYYMMDD', 'HH', 'Zonal price', 'System load', 'Zonal load', 'Day of the week'])
    df_['price'] = df['Zonal price']
    return df_

filename = '/Users/sebastiansuwada/Desktop/Predictive Analytics/Naive_probabilistic_forecasts_List_3/GEFCOM.txt'
df = import_data(filename)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25968 entries, 0 to 25967
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   price   25968 non-null  float64
dtypes: float64(1)
memory usage: 203.0 KB


Obtain Naive forecast for prepare of Point Forcast

In [13]:
def CombinationNaiveForecast(df_1, shift1, shift2):

    # P_w = (P_(w-1)+P_(w-3))/2
    naiv1 = pd.DataFrame()
    naiv1= df_1['price'].shift(shift1)
    naiv2 = pd.DataFrame()
    naiv2= df_1['price'].shift(shift2)
    
    naiv1 = naiv1.dropna()
    naiv2 = naiv2.dropna()
    naiv1.reset_index(drop=True, inplace=True)
    naiv2.reset_index(drop=True, inplace=True)

    min_size = min(len(naiv1), len(naiv2), len(df_1))
    naiv1 = naiv1.tail(min_size)
    naiv2 = naiv2.tail(min_size)
    df_sized = df_1.tail(min_size).copy()

    dfs = [naiv1, naiv2, df_sized]
    for df in dfs:
        df.reset_index(drop=True, inplace=True)

    naivSum = pd.DataFrame()
    naivSum['price'] = (naiv1 + naiv2)/2

    return naivSum, df_sized

shift1 = 24
shift2 = 168
df_sized, naivSum = CombinationNaiveForecast(df, shift1, shift2)
print(naivSum)


        price
0       49.22
1       42.69
2       40.33
3       39.81
4       39.38
...       ...
25795  113.92
25796  107.26
25797   89.02
25798   85.40
25799   86.13

[25800 rows x 1 columns]


Now obtained point forecast can be used for Quantile Regression the dataFrame called naivSum now is our new data that we are going to work on

In [36]:
def RegressionModel(df):
    errors = []
    T = 364

    for h in range(24):
        p_hour = df.loc[h::24, 'price'].values
        h_errors = []

        for day in range(T, len(p_hour)):
            cal_data = p_hour[:T]

            Y = cal_data[1:]  # Y is the next day's prices
            X1 = cal_data[:-1]  # X1 is today's prices
            X0 = np.ones(np.size(X1))

            X = np.stack([X0, X1], axis=1)
            betas = np.dot(np.linalg.inv(np.dot(X.T, X)), np.dot(X.T, Y))
            
            # Forecast for the next day
            X_fut = np.array([1, cal_data[-1]])
            forecast = np.dot(X_fut, betas)

            # Real value for the next day
            real = p_hour[day]

            # Calculate the error
            err = real - forecast

            # Condition for choosing the error formula
            if forecast < real:
                err = err * 0.05 
            else:
                err = err * 0.95

            h_errors.append(err)
    
    print(h_errors)
            # Jak mamy error jak różnicę to zapisujemy


# Split data into half

naivSumTrain = naivSum['price'][:int(len(naivSum)/2)]
naivSumTrain.info()

RegressionModel(naivSum)


<class 'pandas.core.series.Series'>
RangeIndex: 12900 entries, 0 to 12899
Series name: price
Non-Null Count  Dtype  
--------------  -----  
12900 non-null  float64
dtypes: float64(1)
memory usage: 100.9 KB
[-5.234543396021321, -7.305543396021321, -4.740543396021321, -3.6955433960213204, -2.91654339602132, -1.9760433960213215, 0.028497715998877737, 0.7584977159988778, 1.5034977159988778, 0.652497715998878, 0.691497715998878, 0.5224977159988778, 0.12699771599887805, 0.3714977159988777, 0.443497715998878, -2.44154339602132, -3.06854339602132, -1.8525433960213191, -1.957043396021322, -3.125543396021319, -4.32254339602132, -2.8120433960213207, -3.0495433960213205, -2.37504339602132, -3.8380433960213187, -6.65004339602132, -6.05154339602132, -4.88304339602132, -3.059043396021322, -5.05404339602132, -5.59554339602132, -4.389043396021321, -5.557543396021321, -4.142043396021319, -4.66454339602132, -1.3870433960213173, 0.3384977159988779, 0.10699771599887775, -5.27254339602132, -5.7095433960213